# Análisis del riesgo de incumplimiento de los prestatarios

Tu proyecto consiste en preparar un informe para la división de préstamos de un banco. Deberás averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

Tu informe se tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

[En este cuaderno se te brindan pistas, breves instrucciones y sugerencias para pensar. No los ignores, ya que están diseñados para equiparte con la estructura del proyecto y te ayudarán a analizar lo que estás haciendo en un nivel más profundo. Antes de enviar tu proyecto, asegúrate de eliminar todas las sugerencias y descripciones que se te hayan proporcionado. Más bien, haz que este informe parezca como si se lo estuvieras enviando a tus compañeros de equipo para demostrar tus hallazgos: ¡no deben saber que recibiste ayuda externa de nuestra parte! Para ayudarte, hemos colocado las pistas que debes eliminar entre corchetes.]

[Antes de sumergirte en el análisis de tus datos, explica los propósitos del proyecto y las hipótesis que vas a evaluar.]

# Propósito de este proyecto

La intención de este análisis, es determinar si ciertas condiciones socioeconómicas (estado civil y número de hijos)
son determinantes en el comportamiento de pago de los clientes de un banco. Esto con la finalidad de crear una puntuación de crédito para un cliente potencial, la cual se utilizará para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

Para ello, analizaremos los datos con el propósito de responder a las siguientes preguntas:
- ¿Hay alguna conexión entre tener hijos y pagar un préstamo a tiempo?
- ¿Existe una conexión entre el estado civil y el pago a tiempo de un préstamo?
- ¿Existe una conexión entre el nivel de ingresos y el pago a tiempo de un préstamo?
- ¿Cómo afectan los diferentes propósitos del préstamo al reembolso a tiempo del préstamo?

El proyecto consistirá en cuatro etapas:
 1. Exploración de los datos
 2. Preprocesamiento de datos
 3. Prueba de hipótesis
 4. Conclusiones

## Abrir el archivo de datos y ver la información general. <a id='data_review'></a>

Comenzamos con la importación de las librerías y la carga de los datos.

In [1]:
# Cargar todas las librerías
import pandas as pd

# Carga los datos
df = pd.read_csv("/datasets/credit_scoring_eng.csv")
df.head(10)

#Comentario: Usé el método head() para comprobar que se cargaron los datos correctamente

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


## Exploración de datos <a id='data_preprocessing'></a>

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Vamos a mostrar las quince primeras filas

In [6]:
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


El método info(), indica que la tabla tiene 21.525 filas y 12 columnas.

Respecto a los valores ausentes, se aprecia que las columnas `days_employed` y `total_income` son las únicas que tienen valores ausentes, y por alguna razón son la misma cantidad (ambas tienen 19351 entradas non-null). Esto debe revisarse para encontrar el motivo y elegir la mejor opción para solucionarlo. Al mostrar las primeras 15 filas, se ve que los valores ausentes están con el valor `NaN`.

La columna `days_employed` tiene valores negativos. También se debe encontrar la causa y solucionarlo.

Vamos a determinar la cantidad de valores ausentes con el método isnull() más el método sum()

In [7]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Efectivamente, hay la misma cantidad de valores ausentes en las columnas `days_employed` y `total_income`. Investiguemos más a detalle esto para ver a que se debe. Vamos a contar los valores ausentes en todas las filas en las columnas de `days_employed` y `total_income` para confirmar que las muestras que faltan son del mismo tamaño.

In [8]:
import pandas as pd
# Pasamos el argumento keep_default_na=False al DataFrame para convertir los valores ausentes en cadenas vacías
df = pd.read_csv("/datasets/credit_scoring_eng.csv", keep_default_na=False)

# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.
df.loc[(df["days_employed"] == "") & (df["total_income"] == "")]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,,65,secondary education,1,civil partnership,1,M,retiree,0,,to have a wedding
26,0,,41,secondary education,1,married,0,M,civil servant,0,,education
29,0,,63,secondary education,1,unmarried,4,F,retiree,0,,building a real estate
41,0,,50,secondary education,1,married,0,F,civil servant,0,,second-hand car purchase
55,0,,54,secondary education,1,civil partnership,1,F,retiree,1,,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,,47,Secondary Education,1,married,0,M,business,0,,purchase of a car
21495,1,,50,secondary education,1,civil partnership,1,F,employee,0,,wedding ceremony
21497,0,,48,BACHELOR'S DEGREE,0,married,0,F,business,0,,building a property
21502,1,,42,secondary education,1,married,0,F,employee,0,,building a real estate



**Conclusión intermedia**

El número de filas en la tabla filtrada coincide con el número de valores ausentes. Gracias al resultado de filtrar por múltiples condiciones, podemos afirmar que los valores ausentes son simétricos, ya que en las filas donde la columna `days_employed` tiene un valor ausente, también lo tiene la `columna total_income`

Calculemos el porcentaje de valores ausentes en cada columna. Como en en ambas el valor es el mismo ,utilizaremos una sola variable.

In [9]:
total_values = 21525
missing_values = 2174

missing_values_rate = missing_values / total_values

f"El porcentaje de valores ausentes es: {missing_values_rate:.0%}"

'El porcentaje de valores ausentes es: 10%'

Podemos ver que en cada columna, tenemos un 10% de valores ausentes. Esto representa una cantidad significativa, así que buscaremos la mejor forma de completarlos para que no afecte nuestros resultados. Definamos primero si los datos ausentes podrían deberse a alguna característica específica del cliente, como el tipo de empleo u otra cosa

Vamos a investigar a los clientes que no tienen datos en las columnas que tienen valores ausentes.
Primero, creamos una tabla filtrada solo con los valores asuentes y otra sin los valores ausentes

In [12]:
df_filtered_miss = df.loc[(df["days_employed"] == "") & (df["total_income"] == "")]
df_filtered_not_miss = df.loc[(df["days_employed"] != "") & (df["total_income"] != "")]

A continuación, creamos variables para comparar el porcentaje que cada categoría ocupa con y sin datos faltantes, entregando un resultado con valores relativos para comprobar la distribución.

In [14]:
dist_children = df_filtered_not_miss["children"].value_counts(normalize=True)
dist_miss_children = df_filtered_miss["children"].value_counts(normalize=True)

dist_debt = df_filtered_not_miss["debt"].value_counts(normalize=True)
dist_miss_debt = df_filtered_miss["debt"].value_counts(normalize=True)

dist_fam = df_filtered_not_miss["family_status"].value_counts(normalize=True)
dist_fam_miss = df_filtered_miss["family_status"].value_counts(normalize=True)

print(dist_children)
print()
print(dist_miss_children)
print()
print(dist_debt)
print()
print(dist_miss_debt)
print()
print(dist_fam)
print()
print(dist_fam_miss)
print()

dist_children_dif = dist_miss_children - dist_children
print(dist_children_dif)

dist_debt_dif = dist_debt - dist_miss_debt
print(dist_debt_dif)

dist_fam_dif = dist_fam_miss - dist_fam
print(dist_fam_dif)

 0     0.656814
 1     0.224433
 2     0.095654
 3     0.015193
 20    0.003462
-1     0.002274
 4     0.001757
 5     0.000413
Name: children, dtype: float64

 0     0.661914
 1     0.218491
 2     0.093836
 3     0.016559
 20    0.004140
 4     0.003220
-1     0.001380
 5     0.000460
Name: children, dtype: float64

0    0.918816
1    0.081184
Name: debt, dtype: float64

0    0.921803
1    0.078197
Name: debt, dtype: float64

married              0.575836
civil partnership    0.193013
unmarried            0.130484
divorced             0.055966
widow / widower      0.044701
Name: family_status, dtype: float64

married              0.568997
civil partnership    0.203312
unmarried            0.132475
divorced             0.051518
widow / widower      0.043698
Name: family_status, dtype: float64

-1    -0.000894
 0     0.005100
 1    -0.005942
 2    -0.001818
 3     0.001366
 4     0.001463
 5     0.000047
 20    0.000677
Name: children, dtype: float64
0   -0.002988
1    0.002988
Name: d

**Posibles razones por las que hay valores ausentes en los datos**

Se aprecia en la distribución que es similar el porcentaje que ocupa cada categoría con y sin datos faltantes.
Para llegar a esta conclusión, se analizaron los valores en las columnas a las que se hace mención en las hipótesis (`children`, `debt` y `family_status`) para comparar el porcentaje que cada categoría ocupa con y sin datos faltantes. Como se aprecia en la celda anterior, la diferencia entre categorías con y sin valores ausentes es muy pequeña, llegando la mayor variación apenas al 1%.

Según este análisis, los valores ausentes parecen ser aleatorios, pero veamos otras variables para tratar de encontrar un patrón menos evidente

Comprobemos la distribución en el DataFrame entero

In [9]:
df_dist_chil = df["children"].value_counts(normalize=True)
df_dist_fam = df["family_status"].value_counts(normalize=True)
df_dist_debt = df["debt"].value_counts(normalize=True)

print(df_dist_chil)
print()
print(df_dist_fam)
print()
print(df_dist_debt)


 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64

married              0.575145
civil partnership    0.194053
unmarried            0.130685
divorced             0.055517
widow / widower      0.044599
Name: family_status, dtype: float64

0    0.919117
1    0.080883
Name: debt, dtype: float64


**Conclusión intermedia**

La distribución en el conjunto de datos original es prácticamente la misma a la distribución de las tablas filtrada. Eso hace pensar que los valores ausentes son aleatorios

**Conclusiones**

Después de probar distintas variables, no pudimos encontrar ningún patrón subyacente que haga pensar que la falta de datos se debe a algo en específico, por lo que comenzaremos a trabajar con los datos buscando y eliminando duplicados, revisando y corrigiendo valores problemáticos, rellenando los valores ausentes y finalmente agrupando los datos para responder a las hipótesis.

## Transformación de datos

Vamos a repasemos cada columna para ver qué problemas podemos tener en ellas

Como se mencionó anteriormente, comenzaremos con la eliminación de duplicados y la corrección de la información en la columna education.

Veamos todos los valores en la columna de `education` para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente

In [12]:
df["education"].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

Podemos ver que realmente son solo 5 categorías las que hay en esta columna, pero están duplicadas con valores en mayusculas y minúsculas. Vamos a unificarlos colocádolos todos en minúscula.

In [16]:
df['education'] = df['education'].str.lower()

Comprobamos todos los valores en la columna para asegurarnos de están corregidos.

In [17]:
df["education"].value_counts()

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

Ahora, vamos a comprobar los datos de la columna `children`. Revisemos la distribución de los valores en la columna.

In [18]:
df["children"].value_counts(normalize=True)

 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64

Podemos apreciar un valor extraño en esta columna, ya que muestra a 47 personas con -1 hijo. Esto representa solo el 0.21% de la distribución, por lo que se reasignarán las filas con calor -1 a las filas con valor 1. Tomamos esta decisión, ya que en la tabla filtrada con valores ausentes, es menor el porcentaje de personas con un hijo que en la tabla filtrada sin valores ausentes, lo que nos hace pensar que esos valores negativos corresponden realmente a personas con un hijo, ya que de esta forma la distribución entre ambas tablas queda más pareja.

Corrijamos los datos según esa decisión y revisemos la columna para asegurarnos de que está arreglado.

In [20]:
df.loc[df["children"] == -1, "children"] = 1
df["children"].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Revisemos los datos en la columna `days_employed` para identificar posibles valores problemáticos.

In [23]:
df["days_employed"] = pd.to_numeric(df["days_employed"], errors='coerce')
df["days_employed"].value_counts()

-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-7120.517564    1
-2146.884040    1
-881.454684     1
-794.666350     1
-3382.113891    1
Name: days_employed, Length: 19351, dtype: int64

Vamos un gran número de valores negativos, lo cual no tiene sentido. Esto puede deberse a un problema técnico, ya que estos valores deben ser positivos. Por ese motivo, vamos a asumir que los números que se muestran como negativos, deben ser positivos. Así que vamos a convertirlos a positivo usando el método abs() y comprobaremos el resultado para asegurarnos de que esté arreglado.

In [25]:
df["days_employed"] = df["days_employed"].abs()
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.17,purchase of the house
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.15,purchase of the house for my family


Ordenamos la columna de forma ascendente para verificar que ya no hay valores negativos.

In [28]:
df["days_employed"].value_counts().sort_index()

24.141633        1
24.240695        1
30.195337        1
33.520665        1
34.701045        1
                ..
401663.850046    1
401674.466633    1
401675.093434    1
401715.811749    1
401755.400475    1
Name: days_employed, Length: 19351, dtype: int64

Corregidos los valores negativos, vemos que hay números muy grandes que no tienen sentido, ya que al convertirlos en años (dividiendo entre 365) algunos indican una experiencia laboral de más de mil años. Vamos a calcular cuánto podría ser la experiencia laboral máxima considerando la edad de retiro. Según la información obtenida en https://www.creditosenusa.com/a-que-edad-se-jubilan-las-personas-en-estados-unidos/, la edad más común de retiro en USA es de 65 años, y la edad para comenzar a trabajar legalmente es de 16 años.

Calculemos entonces la cantidad máxima de días que podría trabajar una persona, y usemos ese resultado para sustituir los valores que lo superen

In [29]:
# Creamos la variable
max_days_employed = (65 - 16) * 365

# Cambiamos los valores problemáticos
df.loc[df['days_employed'] > 17885, 'days_employed'] = max_days_employed

# Probamos que funcionó
df["days_employed"].max()

17885.0

Revisemos la columna `dob_years` en busca de algún valor sospechoso

In [33]:
df["dob_years"].value_counts().sort_index()

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [49]:
media = df["dob_years"].mean()
mediana = df["dob_years"].median()

print(f"La media de dob_years es: {media}")
print(f"La mediana de dob_years es: {mediana}")

La media de dob_years es: 43.29337979094077
La mediana de dob_years es: 42.0


Encontramos 101 registros con una edad de cero años. Para corregirlos, se calculará la mediana de la columna y se asignará ese valor a los que tengan el parámetro 0. Elegimos hacerlo con la mediana ya que la diferencia entre los valores mínimos y máximos es alta. Además la mediana es 42 y la media 43.29, por lo que tempoco hay una diferencia significativa entre ellas.

Resolvemos los problemas en la columna `dob_years` y comprobamos el resultado.

In [54]:
for row in df:
    dob_years_median = int(df["dob_years"].median())
    df["dob_years"] = df["dob_years"].replace(0, dob_years_median)
    
df["dob_years"].value_counts().sort_index()

19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    614
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

Ahora, revisemos la columna `family_status`.

In [55]:
# Veamos los valores de la columna
df["family_status"].value_counts()

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

No vemos problemas en esta columna

Veamos ahora la columna `gender`.

In [56]:
df["gender"].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Podemos ver un valor atípico en el género, que se muestra como XNA. Investiguémoslo más a fondo

In [59]:
df.loc[df.loc[:,'gender'] == "XNA"]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,some college,2,civil partnership,1,XNA,business,0,32624.825,buy real estate


Vemos que el valor XNA pertenece a la fila 10701, y podemos ver la información de sus otras columnas. Busquemos algún patrón.

In [64]:
# Convertimos los valores de `total_income` a numérico, para poder crear una tabla dinámica.
df["total_income"] = pd.to_numeric(df["total_income"], errors='coerce')

#Hacemos la tabla ordenando las filas por género y nivel de educación, con la edad como columna y los ingresos como valores
df.pivot_table(index=['gender', "education"], columns='dob_years', values='total_income', aggfunc='mean')

dob_years                             19            20            21  \
gender education                                                       
F      bachelor's degree             NaN  15119.561200  27708.936667   
       graduate degree               NaN           NaN           NaN   
       primary education             NaN           NaN  17781.366000   
       secondary education  16409.592667  20917.547133  17376.760100   
       some college         19563.199250  15029.169200  21626.268000   
M      bachelor's degree             NaN  15162.318500  31030.319667   
       graduate degree               NaN           NaN           NaN   
       primary education             NaN           NaN           NaN   
       secondary education  14817.591000  23545.465389  23805.700040   
       some college         14575.717000  16020.785000  28081.110429   
XNA    some college                  NaN           NaN           NaN   

dob_years                             22            23            24  \
gender education                                                       
F      bachelor's degree    22054.250708  27815.240241  21561.647750   
       graduate degree               NaN           NaN           NaN   
       primary education             NaN  25488.916000           NaN   
       secondary education  18749.254082  17921.583371  21450.322976   
       some college         18406.446059  19931.715917  20783.238375   
M      bachelor's degree    34112.762750  24917.811556  30682.198375   
       graduate degree               NaN           NaN           NaN   
       primary education    27119.024000  28561.749000  23388.807000   
       secondary education  25488.063000  22632.666719  25570.040282   
       some college         24908.701143  25389.028526  37230.994667   
XNA    some college                  NaN           NaN  32624.825000   

dob_years                             25            26            27  \
gender education                                                       
F      bachelor's degree    26255.105850  25416.756134  29956.839726   
       graduate degree               NaN           NaN           NaN   
       primary education    29545.078000           NaN           NaN   
       secondary education  20509.704000  20036.676663  22535.838545   
       some college         23414.981500  24398.470900  25576.931042   
M      bachelor's degree    28942.232000  35049.759436  38439.074500   
       graduate degree               NaN           NaN           NaN   
       primary education    28554.684500  27933.828500  29603.175250   
       secondary education  26772.736387  26022.735314  28028.858634   
       some college         30906.532200  28845.914200  33589.093545   
XNA    some college                  NaN           NaN           NaN   

dob_years                             28  ...            66            67  \
gender education                          ...                               
F      bachelor's degree    29149.280698  ...  30167.811375  27101.187565   
       graduate degree               NaN  ...           NaN           NaN   
       primary education    10943.017000  ...  18872.167000  14228.745000   
       secondary education  20892.017392  ...  20291.011394  18361.247815   
       some college         21030.839500  ...  28993.934000  33385.647500   
M      bachelor's degree    35383.542088  ...  22244.844333  54046.783167   
       graduate degree               NaN  ...           NaN           NaN   
       primary education    30389.359250  ...  24027.683500           NaN   
       secondary education  27287.762391  ...  18468.466375  23446.639200   
       some college         27161.495125  ...           NaN           NaN   
XNA    some college                  NaN  ...           NaN           NaN   

dob_years                             68            69            70  \
gender education                                                       
F      bachelor's degree    29586.208900  1

Según el resultado de la tabla dinámica, el valor XNA tiene en la categoría de educación el nivel some college y la edad de 24 años. En este grupo, los ingresos se acercan más a la media del ingreso de los hombres. Por esta razón, cambiaremos ese valor a M y comprobaremos que se haya corregido el problema.

In [66]:
df["gender"] = df["gender"].replace("XNA", "M")
df["gender"].value_counts()

F    14236
M     7289
Name: gender, dtype: int64

Revisemos ahora los valores de la columna `income_type`.

In [67]:
df["income_type"].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

No vemos valores problemáticos en esta columna

Una vez revisados todos los valores, veamos si hay duplicados en nuestros datos con la combinación de los métodos `duplicated()` y `sum()`.

In [68]:
df.duplicated().sum()

71

Eliminemos los duplicados con el método `drop_duplicates()` restableciendo el índice con `reset_index(drop=True)`, y comprobemos si funcionó

In [71]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

Comprobemos el tamaño del DataFrame después de haber ejecutado estas primeras manipulaciones.

In [72]:

df.shape

(21454, 12)

Pasamos de 21525 a 21454 filas. Se corrigieron detalles como el estilo de datos, datos con valores negativos y valores duplicados. Ahora, la información del DataFrame está más limpia y mejor ordenada para seguir con el procesamiento.

# Trabajar con valores ausentes

### Restaurar valores ausentes en `total_income`

Comenzaremos restaurando los valores ausentes de la columna `total_income`. Para eso, crearemos primero una categoría de edad para los clientes, y a continuación haremos una nueva columna con la categoría de edad. De esta forma podemos simplificar la tarea de calcular valores para el ingreso total.


Escribamos una función que calcule la categoría de edad, y comprobemos que funciona.

In [74]:
def assign_age_group(age):
    if age < 20:
        return '10-19'
    elif age < 30:
        return '20-29'
    elif age < 40:
        return '30-39'
    elif age < 50:
        return '40-49'
    elif age < 60:
        return '50-59'
    elif age < 70:
        return '60-69'
    else:
        return "70+"

assign_age_group(19)  

'10-19'

Ahora, creemos una nueva columna basada en la función y comprobemos sus valores.

In [79]:
df["age_group"] = df["dob_years"].apply(assign_age_group)
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,17885.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-29
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40-49
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,50-59
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,30-39
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40-49


Los factores que pueden influir en los singresos son el grado de educación, la edad y el género, por lo que vamos a investigarlos para decidir la forma de completar los valores ausentes.

Primero. creemos una tabla que solo tenga datos sin valores ausentes, con la finalidad de utilizarlos para restaurar la información. Mostraremos algunas de sus filas para comprobar que funciona

In [99]:
df_filtered_not_miss = df.dropna()
df_filtered_not_miss.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,17885.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-29
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40-49
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,50-59
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,30-39
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40-49


In [104]:
# Cambiamos el tipo de dato de la columna age_group a numérico para poder trabajarlo en una tabla dinámica
df_filtered_not_miss.loc[:,"age_group"] = pd.to_numeric(df_filtered_not_miss.loc[:,"age_group"], errors='coerce')
# Comprobamos que funciona
df_filtered_not_miss.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          19351 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         19351 non-null  int64  
 3   education         19351 non-null  object 
 4   education_id      19351 non-null  int64  
 5   family_status     19351 non-null  object 
 6   family_status_id  19351 non-null  int64  
 7   gender            19351 non-null  object 
 8   income_type       19351 non-null  object 
 9   debt              19351 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           19351 non-null  object 
 12  age_group         0 non-null      float64
dtypes: float64(3), int64(5), object(5)
memory usage: 2.1+ MB


Examinemos los valores mínimos de los ingresos según el nivel de educación, género y edad.

In [108]:
df_filtered_min_pivot = df_filtered_not_miss.pivot_table(
    index=["education", "gender"],
    columns="age_group",
    values="total_income",
    aggfunc='min',
    margins=True
)

df_filtered_min_pivot

age_group                       10-19      20-29      30-39      40-49  \
education           gender                                               
bachelor's degree   F             NaN   5820.374   7157.482   6023.274   
                    M             NaN   6844.452   7073.155   8207.823   
graduate degree     F             NaN        NaN  17822.757        NaN   
                    M             NaN        NaN  18551.846  31771.321   
primary education   F             NaN  10707.670  11233.642   5863.853   
                    M             NaN  12856.068  10533.332   5837.099   
secondary education F        9459.851   4494.861   5037.321   4036.463   
                    M       14700.281   6227.210   3392.845   6574.414   
some college        F       12125.986   8154.029  11427.471   8199.590   
                    M       14575.717   8514.371   8163.589   5514.581   
All                          9459.851   4494.861   3392.845   4036.463   

age_group                       50-59      60-69        70+        All  
education           gender                                              
bachelor's degree   F        5148.514   6029.458   5622.079   5148.514  
                    M        7162.598   8023.875  16204.713   6844.452  
graduate degree     F             NaN  40868.031        NaN  17822.757  
                    M       42945.794  15800.399        NaN  15800.399  
primary education   F        4049.374   5531.204   5443.908   4049.374  
                    M        6124.012   8348.851  38032.546   5837.099  
secondary education F        3306.762   3503.298   4812.103   3306.762  
                    M        3418.824   3471.216   3595.641   3392.845  
some college        F        5831.255  13380.093   7327.981   5831.255  
                    M       12654.031  19633.075  19946.795   5514.581  
All                          3306.762   3471.216   3595.641   3306.762

Examinemos los valores máximos de los ingresos según el nivel de educación, género y edad.

In [111]:
df_filtered_max_pivot = df_filtered_not_miss.pivot_table(
    index=["education", "gender"],
    columns="age_group",
    values="total_income",
    aggfunc='max',
    margins=True
)

df_filtered_max_pivot

age_group                       10-19       20-29       30-39       40-49  \
education           gender                                                  
bachelor's degree   F             NaN  112084.059  205804.960  228469.514   
                    M             NaN  113230.059  352136.354  362496.645   
graduate degree     F             NaN         NaN   17822.757         NaN   
                    M             NaN         NaN   18551.846   31771.321   
primary education   F             NaN   65263.983   43728.989   44077.710   
                    M             NaN   60576.988   39349.506   78410.774   
secondary education F       21114.762   88961.881  198426.400  113462.764   
                    M       14934.901  105400.683  276204.162  255618.158   
some college        F       26753.823   62935.872   91982.619  153349.533   
                    M       14575.717  131588.163   81005.434   98963.640   
All                         26753.823  131588.163  352136.354  362496.645   

age_group                        50-59       60-69        70+         All  
education           gender                                                 
bachelor's degree   F       195686.797  117616.523  57508.032  228469.514  
                    M       176552.869  216039.297  38247.228  362496.645  
graduate degree     F              NaN   40868.031        NaN   40868.031  
                    M        42945.794   15800.399        NaN   42945.794  
primary education   F        47385.648   45488.263  32186.898   65263.983  
                    M        39033.628   36156.248  38032.546   78410.774  
secondary education F       109994.128  274402.943  54160.801  274402.943  
                    M       175672.758   85419.348  41398.448  276204.162  
some college        F        63041.599   65981.816  14479.193  153349.533  
                    M        85636.260   54236.297  19946.795  131588.163  
All                         195686.797  274402.943  57508.032  362496.645

Examinemos los valores medios de los ingresos según el nivel de educación, género y edad.

In [112]:
df_filtered_mean_pivot = df_filtered_not_miss.pivot_table(
    index=["education", "gender"],
    columns="age_group",
    values="total_income",
    aggfunc='mean',
    margins=True
)

df_filtered_mean_pivot

age_group                          10-19         20-29         30-39  \
education           gender                                             
bachelor's degree   F                NaN  27125.356023  30837.188327   
                    M                NaN  33762.764003  40663.225319   
graduate degree     F                NaN           NaN  17822.757000   
                    M                NaN           NaN  18551.846000   
primary education   F                NaN  23193.671000  21351.115667   
                    M                NaN  29445.893944  22393.454244   
secondary education F       16409.592667  20576.754464  23098.648252   
                    M       14817.591000  26592.616928  29468.000124   
some college        F       19563.199250  22252.881988  29429.499836   
                    M       14575.717000  30003.804955  35848.935671   
All                         16993.942462  25572.630177  28312.479963   

age_group                          40-49         50-59         60-69  \
education           gender                                             
bachelor's degree   F       31996.687329  31606.996179  28276.774677   
                    M       43162.533497  38050.559168  35283.789367   
graduate degree     F                NaN           NaN  40868.031000   
                    M       31771.321000  42945.794000  15800.399000   
primary education   F       21321.173348  17261.786325  18124.956956   
                    M       25731.594440  19445.126615  20467.500667   
secondary education F       23731.202535  22867.157726  21478.446092   
                    M       30527.046860  27335.852349  22456.058661   
some college        F       32123.089000  24660.281568  28915.868647   
                    M       36242.256750  35454.835143  34898.652833   
All                         28491.929026  25811.700327  23242.812818   

age_group                            70+           All  
education           gender                              
bachelor's degree   F       25604.600563  30306.441576  
                    M       27472.424429  38981.070503  
graduate degree     F                NaN  29345.394000  
                    M                NaN  27267.340000  
primary education   F       15702.942667  19118.479588  
                    M       38032.546000  23798.931664  
secondary education F       18554.790650  22671.099805  
                    M       21801.537667  28296.294264  
some college        F       10903.587000  26470.312199  
                    M       19946.795000  33207.574702  
All                         20125.658331  26787.568355

Examinemos los valores míedianos de los ingresos en función de los factores que identificamos

In [113]:
# Examina los valores medianos de los ingresos en función de los factores que identificaste
df_filtered_median_pivot = df_filtered_not_miss.pivot_table(
    index=["education", "gender"],
    columns="age_group",
    values="total_income",
    aggfunc='median',
    margins=True
)

df_filtered_median_pivot

age_group                       10-19       20-29       30-39      40-49  \
education           gender                                                 
bachelor's degree   F             NaN  23481.1275  26205.4365  27571.074   
                    M             NaN  30717.2190  33862.4190  35145.750   
graduate degree     F             NaN         NaN  17822.7570        NaN   
                    M             NaN         NaN  18551.8460  31771.321   
primary education   F             NaN  17781.3660  18962.6750  21215.459   
                    M             NaN  27933.8285  19810.2530  21887.825   
secondary education F       17954.841  18130.7060  20645.4560  21133.000   
                    M       14817.591  24662.6090  26244.6280  26834.295   
some college        F       19686.494  19925.4750  25131.3105  28377.384   
                    M       14575.717  26768.9370  34596.9060  30746.077   
All                         14934.901  22799.2580  24667.5280  24755.696   

age_group                        50-59       60-69         70+         All  
education           gender                                                  
bachelor's degree   F       27112.8490  24754.4010  24479.0300  26063.4715  
                    M       32057.2270  28020.4230  25497.3920  32675.8355  
graduate degree     F              NaN  40868.0310         NaN  29345.3940  
                    M       42945.7940  15800.3990         NaN  25161.5835  
primary education   F       16220.4855  16400.0270  14065.3190  17223.9615  
                    M       18767.1200  18402.1400  38032.5460  21204.0860  
secondary education F       20156.6200  18458.0820  18146.7015  20101.2700  
                    M       24270.0410  20299.6160  19972.8130  25435.5815  
some college        F       20531.8480  28178.9170  10903.5870  22836.0820  
                    M       29405.0380  32316.3535  19946.7950  29975.7245  
All                         22203.0745  19817.4400  18751.3240  23202.8700

Estas cuatro tablas dinámicas se pueden unificar de la siguiete manera con el método agg()

In [52]:
df_grouped = df_filtered_not_miss.groupby(['education', "age_group", "gender"]).agg(
{"total_income": ["min", 'max', "mean", "median"]})
df_grouped

total_income                            \
                                              min         max          mean   
education           age_group gender                                          
bachelor's degree   20-29     F          5820.374  112084.059  27125.356023   
                              M          6844.452  113230.059  33762.764003   
                    30-39     F          7157.482  205804.960  30837.188327   
                              M          7073.155  352136.354  40663.225319   
                    40-49     F          6023.274  228469.514  31996.687329   
                              M          8207.823  362496.645  43162.533497   
                    50-59     F          5148.514  195686.797  31606.996179   
                              M          7162.598  176552.869  38050.559168   
                    60-69     F          6029.458  117616.523  28276.774677   
                              M          8023.875  216039.297  35283.789367   
                    70+       F          5622.079   57508.032  25604.600563   
                              M         16204.713   38247.228  27472.424429   
graduate degree     30-39     F         17822.757   17822.757  17822.757000   
                              M         18551.846   18551.846  18551.846000   
                    40-49     M         31771.321   31771.321  31771.321000   
                    50-59     M         42945.794   42945.794  42945.794000   
                    60-69     F         40868.031   40868.031  40868.031000   
                              M         15800.399   15800.399  15800.399000   
primary education   20-29     F         10707.670   65263.983  23193.671000   
                              M         12856.068   60576.988  29445.893944   
                    30-39     F         11233.642   43728.989  21351.115667   
                              M         10533.332   39349.506  22393.454244   
                    40-49     F          5863.853   44077.710  21321.173348   
                              M          5837.099   78410.774  25731.594440   
                    50-59     F          4049.374   47385.648  17261.786325   
                              M          6124.012   39033.628  19445.126615   
                    60-69     F          5531.204   45488.263  18124.956956   
                              M          8348.851   36156.248  20467.500667   
                    70+       F          5443.908   32186.898  15702.942667   
                              M         38032.546   38032.546  38032.546000   
secondary education 10-19     F          9459.851   21114.762  16409.592667   
                              M         14700.281   14934.901  14817.591000   
                    20-29     F          4494.861   88961.881  20576.754464   
                              M          6227.210  105400.683  26592.616928   
                    30-39     F          5037.321  198426.400  23098.648252   
                              M          3392.845  276204.162  29468.000124   
                    40-49     F          4036.463  113462.764  23731.202535   
                              M          6574.414  255618.158  30527.046860   
                    50-59     F          3306.762  109994.128  22867.157726   
                              M          3418.824  175672.758  27335.852349   
                    60-69     F          3503.298  274402.943  21478.446092   
                              M          3471.216   85419.348  22456.058661   
                    70+       F          4812.103   54160.801  18554.790650   
                              M          3595.641   41398.448  21801.537667   
some college        10-19     F         12125.986   26753.823  19563.199250   
                              M         14575.717   14575.717  14575.717000   
                    20-29     F          8154.029   62935.872  22252.881988   
                              M          8514.371  131588.163  30003.804955   
        

Se hicieron varias tablas dinámicas en las que se compararon los ingresos, máximos, mínimos, medios y medianos por grupos de edades según el género y el grado de educación, llegando a las siguientes conclusiones. 

- Se aprecia que en todos los rangos de edad, en la categoría bachelor's degree los ingresos de los hombres son mayores a los de las mujeres. Así mismo, los ingresos de ambos sexos disminuyen luego de los 70 años.

- En la categoría graduate degree, en el grupo de edad de 30 a 39 años, los hombres ganan más que las mujeres, pero en el grupo de 60 a 69 años los ingresos de las mujeres casi triplican a los de los hombres.

- En la categoría educación primaria, se repite el patrón que se ve en bachelor's degree: los ingresos de los hombres son sueriores a los de las mujeres. Sin embargo, los ingresos en general de ambos sexos son más bajos con este grado de educación que respecto a bachelor's degree.

- En cuanto a las personas en secondary education, las mujeres de 10 a 19 años tienen ingresos superiores a los de los hombres, situación que se invierte en los demás grupos de edades.

- Por último, en la categoría some college, las mujeres de 10 a 19 años tienen ingresos superiores a los de los hombres, situación que se invierte en los demás grupos de edades. Al igual que en bachelor's degree, los ingresos de ambos sexos disminuyen luego de los 70 años.


Vamos a rellenar los valores ausentes con la mediana, ya que, luego del análisis, se pueden apreciar valores atípicos en el sentido que los mínimos y los máximos tienen una gran diferencia entre si en todos los sexos y rangos de edad, independientemente del nivel de educación.

Comenzaremos escribiendo una función para completar los valores ausentes.

In [116]:
def fill_missing_values(row):
    gender = row["gender"]
    total_income = row["total_income"]
    
    if pd.isna(total_income):
        return df_filtered_not_miss[df_filtered_not_miss['gender']==gender]['total_income'].median()
    return total_income                         

Comprobemos si funciona.

In [117]:
fill_missing_values(df.iloc[12])

26838.552000000003

Vamos a aplicarlo a cada fila.

In [119]:
df["total_income"] = df.apply(fill_missing_values, axis=1)
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,17885.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-29
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40-49
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,50-59
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,30-39
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40-49


Comprobemos si tenemos algún error.

In [121]:
try:
    df["total_income"] = df.apply(fill_missing_values, axis=1)
except:
    print("Error")

In [122]:
df.iloc[12]

children                              0
days_employed                       NaN
dob_years                            65
education           secondary education
education_id                          1
family_status         civil partnership
family_status_id                      1
gender                                M
income_type                     retiree
debt                                  0
total_income                  26838.552
purpose               to have a wedding
age_group                         60-69
Name: 12, dtype: object

¡Genial! No tenemos errores. Vamos a comprobar que el número total de valores en esta columna coincida con el número de valores en otras columnas.

In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  age_group         21454 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


###  Restaurar valores en `days_employed`

Revisemos la información relacionada con esta columna para determinar si debemos usar medias o medianas para reemplazar los valores ausentes. Al igual que en la sección anterior, siplifiquemos una sola tabla con los máximos, mínimos, medias y medianas utilizando el método agg().

In [131]:
df_grouped = df_filtered_not_miss.groupby(['education', "age_group", "gender"]).agg(
{"days_employed": ["min", 'max', "mean", "median"]})
df_grouped

days_employed                \
                                               min           max   
education           age_group gender                               
bachelor's degree   20-29     F          85.299449  17885.000000   
                              M          96.205293   4019.228451   
                    30-39     F          34.701045  17885.000000   
                              M          24.240695   8602.090178   
                    40-49     F          47.109840  17885.000000   
                              M          79.099355  17885.000000   
                    50-59     F          93.746886  17885.000000   
                              M          65.163190  17885.000000   
                    60-69     F         133.939567  17885.000000   
                              M         331.925746  17885.000000   
                    70+       F        6025.506521  17885.000000   
                              M       13923.375086  17885.000000   
graduate degree     30-39     F        5968.075884   5968.075884   
                              M        2351.431934   2351.431934   
                    40-49     M         409.200149    409.200149   
                    50-59     M        5352.038180   5352.038180   
                    60-69     F       17885.000000  17885.000000   
                              M       17885.000000  17885.000000   
primary education   20-29     F         313.762280   1294.277533   
                              M         108.277236   3698.498260   
                    30-39     F         224.704497   6670.000568   
                              M          62.108487  17885.000000   
                    40-49     F         138.365707  17885.000000   
                              M         121.108313   4193.718465   
                    50-59     F         412.490556  17885.000000   
                              M         204.289415  17885.000000   
                    60-69     F        2000.751587  17885.000000   
                              M        1835.480848  17885.000000   
                    70+       F       17885.000000  17885.000000   
                              M       17885.000000  17885.000000   
secondary education 10-19     F         115.876036    793.358581   
                              M         111.762790   1020.183130   
                    20-29     F          61.596443  17885.000000   
                              M          67.036625  17885.000000   
                    30-39     F          24.141633  17885.000000   
                              M          86.924816  17885.000000   
                    40-49     F          50.128298  17885.000000   
                              M          30.195337  17885.000000   
                    50-59     F          58.276481  17885.000000   
                              M          89.163652  17885.000000   
                    60-69     F         100.309421  17885.000000   
                              M         114.047745  17885.000000   
                    70+       F         730.874426  17885.000000   
                              M         486.662226  17885.000000   
some college        10-19     F         509.969922    948.225743   
                              M         885.268574    885.268574   
                    20-29     F          51.496885   3900.636853   
                              M         126.962615   3565.212416   
                    30-39     F          74.995250   6387.136079   
                              M          91.894242   6391.782776   
                    40-49     F         161.987040  17885.000000   
                              M         109.349654  17885.000000   
                    50-59     F         438.934197  17885.000000   
                              M         278.882902  13264.142143   
                    60-69     F         159.143455  17885.000000   
                              M         441.431449  17885.000000   
                    70+  

Como se puede apreciar, los datos tienen valores atípicos significativos, ya que la diferencia entre los valores mínimos y máximos es grande. Por esta razón, utilizaremos la mediana para completar los valores ausentes.

Vamos a escribir una función para eso y a comprobar que funciona.

In [132]:
def calc(row):
    gender = row["gender"]
    days_employed = row["days_employed"]
    
    if pd.isna(days_employed):
        return df_filtered_not_miss[df_filtered_not_miss['gender']==gender]['days_employed'].median()
    return days_employed

calc(df.iloc[12])

1662.9179531733637

Apliquemos la función a todas las filas.

In [134]:
df["days_employed"] = df.apply(calc, axis=1)
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,17885.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-29
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40-49
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,50-59
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,30-39
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40-49


Comprobamos si funcionó revisando que el número total de valores en esta columna coincida con el número de valores en otras columnas.

In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  age_group         21454 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


## Clasificación de datos

Ya rellenamos los valores ausentes. Ahora, vamos a organizar los datos para responder a las preguntas planteadas inicialmente: 
- ¿Hay alguna conexión entre tener hijos y pagar un préstamo a tiempo?
- ¿Existe una conexión entre el estado civil y el pago a tiempo de un préstamo?
- ¿Existe una conexión entre el nivel de ingresos y el pago a tiempo de un préstamo?
- ¿Cómo afectan los diferentes propósitos del préstamo al reembolso a tiempo del préstamo?


Mostremos los valores de los datos que se mencionan en las preguntas para la clasificación.

In [137]:
df_clas = df.groupby(['children', "family_status", "purpose"]).agg(
{"total_income": ["min", 'max', "mean", "median"]})
df_clas

total_income  \
                                                                             min   
children family_status     purpose                                                 
0        civil partnership building a property                          9940.303   
                           building a real estate                       5330.769   
                           buy commercial real estate                   6167.401   
                           buy real estate                              9614.879   
                           buy residential real estate                 11094.820   
...                                                                          ...   
20       unmarried         purchase of the house                        8635.452   
                           transactions with commercial real estate    19529.973   
         widow / widower   housing transactions                         9579.136   
                           purchase of the house                       21464.845   
                           transactions with commercial real estate    42315.974   

                                                                                \
                                                                           max   
children family_status     purpose                                               
0        civil partnership building a property                       65303.806   
                           building a real estate                    51420.423   
                           buy commercial real estate                59839.973   
                           buy real estate                           81208.413   
                           buy residential real estate               88961.881   
...                                                                        ...   
20       unmarried         purchase of the house                      8635.452   
                           transactions with commercial real estate  19529.973   
         widow / widower   housing transactions                      13108.291   
                           purchase of the house                     21464.845   
                           transactions with commercial real estate  42315.974   

                                                                                   \
                                                                             mean   
children family_status     purpose                                                  
0        civil partnership building a property                       26233.609356   
                           building a real estate                    24045.966970   
                           buy commercial real estate                24681.440182   
                           buy real estate                           29124.113600   
                           buy residential real estate               30216.101474   
...                                                                           ...   
20       unmarried         purchase of the house                      8635.452000   
                           transactions with commercial real estate  19529.973000   
         widow / widower   housing transactions                      11343.713500   
                           purchase of the house                     21464.845000   
                           transactions with commercial real estate  42315.974000   

                                                                                 
                                                                         median  
children family_status     purpose                                               
0        civil partnership building a property                       22435.0690  
                           building a real estate                    21479.1970  
                           buy commercial real estate                21464.8450  
                           buy real estate                 

Podemos ver que en la columna purpose, hay muchos valores similares. En vista de eso, vamos a comprobar los valores únicos para ver si podemos simplifiar nuestra tabla. Comprobemos los valores únicos.

In [138]:
df["purpose"].sort_values().unique()

array(['building a property', 'building a real estate',
       'buy commercial real estate', 'buy real estate',
       'buy residential real estate', 'buying a second-hand car',
       'buying my own car', 'buying property for renting out', 'car',
       'car purchase', 'cars', 'construction of own property',
       'education', 'getting an education', 'getting higher education',
       'going to university', 'having a wedding', 'housing',
       'housing renovation', 'housing transactions', 'profile education',
       'property', 'purchase of a car', 'purchase of my own house',
       'purchase of the house', 'purchase of the house for my family',
       'real estate transactions', 'second-hand car purchase',
       'supplementary education', 'to become educated', 'to buy a car',
       'to get a supplementary education', 'to have a wedding',
       'to own a car', 'transactions with commercial real estate',
       'transactions with my real estate', 'university education',
       'we

Podemos identiicar los siguientes grupos comunes:
- Vivienda
- Compra de auto
- Educación
- Boda

Vamos a escribir una función para clasificar los datos en función de temas comunes, para luego poder crear una columna con las categorías agrupadas.

In [141]:
def replace_purposes(row):
    if "property" in row["purpose"] or "real estate" in row["purpose"] or "housing" in row["purpose"] or "house" in row["purpose"]:
        return "Real Estate"
    elif "car" in row['purpose']:
        return 'Car'
    elif "wedding" in row["purpose"]:
        return "Wedding"
    elif "education" or "university" or "educated" in row["purpose"]:
        return 'Education'
    
df["purpose_group"] = df.apply(replace_purposes, axis=1)
df["purpose_group"].value_counts()

Real Estate    10811
Car             4306
Education       4013
Wedding         2324
Name: purpose_group, dtype: int64

Con las categorías agrupadas, veamos si es necesario clasificar los datos numéricos y crear categorías para ellos.

In [142]:
df.groupby("total_income")["debt"].value_counts(normalize=True).sort_index()

total_income  debt
3306.762      1       1.0
3392.845      0       1.0
3418.824      0       1.0
3471.216      0       1.0
3503.298      0       1.0
                     ... 
273809.483    0       1.0
274402.943    0       1.0
276204.162    0       1.0
352136.354    1       1.0
362496.645    0       1.0
Name: debt, Length: 19351, dtype: float64

Obtengamos estadísticas más resumidas para la columna.

In [143]:
df_filtered = df.groupby('debt').agg(
{'total_income': ['max', 'min']}).value_counts()

df_filtered

(total_income, max)  (total_income, min)
352136.354           3306.762               1
362496.645           3392.845               1
dtype: int64

Para simplificar el análisis, vamos a agrupar los ingresos totales en rangos de ingresos. Escribiremos una función para ello y comprobaremos que funciona.

In [155]:
def income_group(row):
    total_income = row["total_income"]
    if total_income < 10000:
        return '0-10000'
    elif total_income < 30000:
        return '10000-30000'
    elif total_income < 50000:
        return '30000-50000'
    elif total_income < 70000:
        return '50000-70000'
    elif total_income < 100000:
        return '70000-100000'
    elif total_income < 150000:
        return '100000-150000'
    else:
        return "150000+"
    
df["income_group"] = df.apply(income_group, axis=1)
df["income_group"].value_counts().sort_index()

0-10000            926
10000-30000      14609
100000-150000       71
150000+             28
30000-50000       4599
50000-70000        942
70000-100000       279
Name: income_group, dtype: int64

Con la agrupación lista, podemos analizar la información con tablas dinámicas.

In [156]:
df_filtered = df.pivot_table(
    index=["debt", "family_status", "purpose_group"],
    columns="income_group",
    values="total_income",
    aggfunc='count',
    margins=True
)

df_filtered

income_group                          0-10000  10000-30000  100000-150000  \
debt family_status     purpose_group                                        
0    civil partnership Car               16.0        257.0            2.0   
                       Education         18.0        224.0            1.0   
                       Real Estate       34.0        603.0            2.0   
                       Wedding           86.0       1453.0            8.0   
     divorced          Car                9.0        169.0            NaN   
                       Education          7.0        167.0            NaN   
                       Real Estate       22.0        409.0            1.0   
     married           Car              109.0       1666.0           10.0   
                       Education        119.0       1623.0            6.0   
                       Real Estate      302.0       4372.0           25.0   
     unmarried         Car               15.0        397.0            2.0   
                       Education         11.0        361.0            NaN   
                       Real Estate       53.0       1008.0            9.0   
     widow / widower   Car               21.0        145.0            NaN   
                       Education         14.0        136.0            1.0   
                       Real Estate       32.0        372.0            NaN   
1    civil partnership Car                2.0         37.0            NaN   
                       Education          NaN         47.0            NaN   
                       Real Estate        1.0         67.0            1.0   
                       Wedding            3.0        146.0            NaN   
     divorced          Car                1.0         16.0            NaN   
                       Education          NaN         11.0            NaN   
                       Real Estate        1.0         34.0            NaN   
     married           Car               12.0        151.0            NaN   
                       Education          5.0        162.0            1.0   
                       Real Estate       22.0        335.0            1.0   
     unmarried         Car                1.0         61.0            NaN   
                       Education          2.0         44.0            1.0   
                       Real Estate        4.0         88.0            NaN   
     widow / widower   Car                2.0         16.0            NaN   
                       Education          1.0         12.0            NaN   
                       Real Estate        1.0         20.0            NaN   
All                                     926.0      14609.0           71.0   

income_group                          150000+  30000-50000  50000-70000  \
debt family_status     purpose_group                                      
0    civil partnership Car                NaN         86.0         18.0   
                       Education          1.0         81.0         14.0   
                       Real Estate        2.0        209.0         40.0   
                       Wedding            2.0        472.0         92.0   
     divorced          Car                NaN         66.0         12.0   
                       Education          NaN         35.0          9.0   
                       Real Estate        3.0        152.0         32.0   
     married           Car                2.0        564.0        116.0   
                       Education          2.0        480.0        117.0   
                       Real Estate       11.0       1418.0        300.0   
     unmarried         Car                1.0        111.0         22.0   
                       Education          NaN        120.0         17.0   
                       Real Estate        2.0        311.0         64.0   
     widow / widower   Car                NaN         27.0          3.0   
                       Education          NaN         32.0          1.0   
                       Real E

Podemos ver que, por mucho, el grupo de ingresos que más solicita algún tipo de financiamiento es el que gana entre 10.000 y 30.000, seguidos por el grupo de ingresos de 30.000 a 50.000.

Ahora, hagamos otra función para clasificar estos grupos según su historial crediticio, diferenciando a los clientes más riesgosos de los menos riesgosos, y crearemos una nueva columna con el resultado de esta función para poder sacar nuestras conclusiones.

In [157]:
def credit_score(row): 
    
    purpose_group = row["purpose_group"]
    debt = row["debt"]
   
    if purpose_group == "Real Estate": 
        if debt == 1:
            return "high risk"
    if purpose_group == "Car":
        if debt == 1:
            return "high risk"
    if purpose_group == "Education":
        if debt == 1:
            return "high risk"
    if purpose_group == "Wedding":
        if debt == 1:
            return "high risk"
    else:
        return "low risk"
    
df["risk_level"] = df.apply(credit_score, axis=1)

Contemos los valores de cada categoría para ver la distribución.

In [158]:
df["risk_level"].value_counts(normalize=True)

low risk     0.909867
high risk    0.090133
Name: risk_level, dtype: float64

Del total de clientes, tenemos que el 91% son de bajo riesgo, y un 9% es más riesgoso. Veámoslo con más detalle probando las hipótesis en la siguiente sección.

## Comprobación de las hipótesis


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

Comprobemos los datos sobre los hijos y los pagos puntuales con una tabla dinámica.

In [180]:
children_risk = df.pivot_table(
    index=["debt", "risk_level"],
    columns="children",
    values="total_income",
    aggfunc='count',
    margins=True
)

df_grouped = df.groupby('children').agg({'debt': ['count', 'sum']})

df_grouped["conversion"] = df_grouped["debt"]["sum"] / df_grouped["debt"]["count"]
print(df_grouped)

children_risk

           debt       conversion
          count   sum           
children                        
0         14091  1063   0.075438
1          4855   445   0.091658
2          2052   194   0.094542
3           330    27   0.081818
4            41     4   0.097561
5             9     0   0.000000
20           76     8   0.105263


,children,0,1,2,3,4,5,20,All
debt,risk_level,,,,,,,,
0,low risk,11613.0,3927.0,1655.0,276.0,34.0,8.0,62.0,17575
1,high risk,1063.0,445.0,194.0,27.0,4.0,NaN,8.0,1741
All,,12676.0,4372.0,1849.0,303.0,38.0,8.0,70.0,19316


**Conclusión**

Podemos ver que la tasa de incumplimiento es directamente proporcional al número de hijos, por lo que se puede establecer no solo que los clientes sin hijos son mejores pagadores, sino que mientras más hijos tenga una persona, mayores son sus posibilidades de no pagar bien un crédito.

**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

Comprobemos los datos del estado familiar y los pagos a tiempo.

In [181]:
family_risk = df.pivot_table(
    index=["debt", "risk_level"],
    columns="family_status",
    values="total_income",
    aggfunc='count',
    margins=True
)

df_grouped = df.groupby('family_status').agg({'debt': ['count', 'sum']})

df_grouped["conversion"] = df_grouped["debt"]["sum"] / df_grouped["debt"]["count"]
print(df_grouped)

family_risk

                    debt      conversion
                   count  sum           
family_status                           
civil partnership   4151  388   0.093471
divorced            1195   85   0.071130
married            12339  931   0.075452
unmarried           2810  274   0.097509
widow / widower      959   63   0.065693


,family_status,civil partnership,divorced,married,unmarried,widow / widower,All
debt,risk_level,,,,,,
0,low risk,1625,1110,11408,2536,896,17575
1,high risk,388,85,931,274,63,1741
All,,2013,1195,12339,2810,959,19316


**Conclusión**

Según el estado civil, podemos ve que los peores pagadores son las personas solteras, seguidas de aquelas con unión civil, las personas casadas, personas divorciadas y personas viudas. De acuerdo a esta información, podemos concluir que los clientes solteros y aquellos con unión civil, son los más riesgosos a la hora de otorgarles un crédito. Los clientes casados, divorciados y viudos son los menos riesgosos, en ese orden.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

Revisemos los datos del nivel de ingresos y los pagos a tiempo

In [175]:
income_risk = df.pivot_table(
    index=["debt", "risk_level"],
    columns="income_group",
    values="total_income",
    aggfunc='count',
    margins=True
)

df_grouped = df.groupby('income_group').agg({'debt': ['count', 'sum']})

df_grouped["conversion"] = df_grouped["debt"]["sum"] / df_grouped["debt"]["count"]
print(df_grouped)

income_risk

                debt       conversion
               count   sum           
income_group                         
0-10000          926    58   0.062635
10000-30000    14609  1247   0.085358
100000-150000     71     4   0.056338
150000+           28     2   0.071429
30000-50000     4599   344   0.074799
50000-70000      942    70   0.074310
70000-100000     279    16   0.057348


,income_group,0-10000,10000-30000,100000-150000,150000+,30000-50000,50000-70000,70000-100000,All
debt,risk_level,,,,,,,,
0,low risk,782,11909,59,24,3783,780,238,17575
1,high risk,58,1247,4,2,344,70,16,1741
All,,840,13156,63,26,4127,850,254,19316


**Conclusión**

Podemos apreciar que los clientes son menos riesgosos a medida que sus ingresos son mayores. Los clientes con ingresos menores a 10.000 se salen del patrón con una tasa de incumplimiento más baja, probablemente porque al ser bajos sus ingresos, no asumen grandes compromisos. En el extremo opuesto, tenemos que la tasa de incumplimiento aumenta en los clientes con ingresos superiores a los 150.000, probablemente porque al ser altos sus ingresos, asumen mayores compromisis financieros. Podemos concluir entonces que el rango de ingresos más riesgoso está entre los 10.000 y 30.000, descendiendo el riego a medida que aumentan los ingresos, y que a partir de los 150.000 en adelante, también hay que analizar bien al cliente, ya que aumenta la tasa de incumplimento.

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

Veamos los porcentajes de tasa de incumplimiento para cada propósito del crédito y analízalos

In [178]:
purpose_risk = df.pivot_table(
    index=["debt", "risk_level"],
    columns="purpose_group",
    values="total_income",
    aggfunc='count',
    margins=True
)
df_grouped = df.groupby('purpose_group').agg({'debt': ['count', 'sum']})

df_grouped["conversion"] = df_grouped["debt"]["sum"] / df_grouped["debt"]["count"]
print(df_grouped)

purpose_risk

                debt      conversion
               count  sum           
purpose_group                       
Car             4306  403   0.093590
Education       4013  370   0.092200
Real Estate    10811  782   0.072334
Wedding         2324  186   0.080034


,purpose_group,Car,Education,Real Estate,Wedding,All
debt,risk_level,,,,,
0,low risk,3903.0,3643.0,10029.0,NaN,17575
1,high risk,403.0,370.0,782.0,186.0,1741
All,,4306.0,4013.0,10811.0,186.0,19316


**Conclusión**

Podemos apreciar que la mayor tasa de incumplimiento es la de los créditos para la compra de vehículos. Es seguida por la de los créditos para estudios, créditos para boda y créditos para inmuebles. En ese sentido, podemos concluir que los clientes menos riesgosos son aquellos que buscan financiaiento para la compra de un inmueble, mientras que los más riesgosos son aquellos que buscan comprar un auto o realizar algún tipo de estudio.

# Conclusión general 

Se procesaron los datos para identificar valores ausentes, duplicados y valores problemáticos, a fin de tener la información lo más limpia posible para el análisis. Una vez hecho esto, se agruparon las variables de interes con el uso de funciones, para hacer más sencilla la clasificación de los datos.

Repondamos entonces las hipótesis planteadas:
- ¿Hay alguna conexión entre tener hijos y pagar un préstamo a tiempo?
Si,la tasa de incumplimiento es directamente proporcional al número de hijos. Los clientes sin hijos son mejores pagadores, y a medida que el número de hijos aumenta, mayor es su tasa de incumplimiento.
- ¿Existe una conexión entre el estado civil y el pago a tiempo de un préstamo?
Si, los clientes solteros y aquellos con unión civil, son los los que tienen peor tasa de incumplimiento. Los clientes casados, divorciados y viudos son los menos riesgosos.
- ¿Existe una conexión entre el nivel de ingresos y el pago a tiempo de un préstamo?
Si, la tasa de incumplimiento más alta está en el rando de ingresos entre los 10.000 y 30.000. A medida que aumentan los ingresos, la tasa de incumplimiento disminuye, rompiéndose el patrón en los ingresos que superan los 150.000.
- ¿Cómo afectan los diferentes propósitos del préstamo al reembolso a tiempo del préstamo?
Los clientes menos riesgosos son aquellos que buscan financiaiento para la compra de un inmueble, mientras que los más riesgosos son aquellos que buscan comprar un auto o realizar algún tipo de estudio.

Podemos finalmente concluir que todos estos factores son importantes y arrojan información valiosa al momento de medir el riesgo de un potencial cliente. Por esta razón, se sugiere hacer uso de todas las variables en conjunto, ya que de esta forma se pueden tener indicadores más completos acerca del nivel de riesgo que representa el perfil de un cliente especifico.